In [14]:
import pandas as pd
import pm4py

In [15]:
dataframe = pd.read_csv('example_log1.csv', sep=',')
print(dataframe["Activity"].unique())
dataframe = pm4py.format_dataframe(dataframe, case_id='Case ID', activity_key='Activity', timestamp_key='Start Date')
event_log = pm4py.convert_to_event_log(dataframe)

['Inbound Call' 'Handle Case' 'Call Outbound' 'Inbound Email'
 'Handle Email' 'Email Outbound']


In [16]:
a = pm4py.discover_bpmn_inductive(event_log)
# pm4py.save_vis_bpmn(a, "bpmn_Example.jpg")
pm4py.write_bpmn(a, "diagram.bpmn")

In [ ]:
import xml.etree.ElementTree as ET

# Define namespaces for easier access
NAMESPACES = {
    'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL',
    'bpmndi': 'http://www.omg.org/spec/BPMN/20100524/DI',
    'dc': 'http://www.omg.org/spec/DD/20100524/DC',
    'di': 'http://www.omg.org/spec/DD/20100524/DI'
}

# register them for better naming
for prefix, uri in NAMESPACES.items():
    ET.register_namespace(prefix, uri)
    
def parse_bpmn_file(file_path):
    """Parses a BPMN XML file and returns the ElementTree root."""
    tree = ET.parse(file_path)
    root = tree.getroot()
    return tree, root

def find_elements_by_tag(root, tag_name):
    """Finds all elements with a given tag name, handling namespaces."""
    # ElementTree's findall requires the full qualified name for namespaced elements
    # {namespace_uri}tag_name
    elements = root.findall(f'.//{NAMESPACES["bpmn"]}{tag_name}')
    return elements

def find_element_by_id(root, element_id):
    """Finds an element by its ID attribute."""
    # Use XPath-like syntax for attribute filtering
    for element in root.iter():
        if 'id' in element.attrib and element.attrib['id'] == element_id:
            return element
    return None

def iterate_and_print_process_elements(root):
    """Iterates over common BPMN process elements and prints their details."""
    print("--- Process Elements ---")
    process = root.find(f'{NAMESPACES["bpmn"]}process')
    if process is None:
        print("No process found.")
        return

    for element in process:
        tag = element.tag.replace(NAMESPACES["bpmn"], '') # Remove namespace prefix for readability
        element_id = element.attrib.get('id', 'N/A')
        element_name = element.attrib.get('name', 'N/A')
        print(f"  Tag: {tag}, ID: {element_id}, Name: {element_name}, Attributes: {element.attrib}")

        # Specific details for certain elements
        if tag == 'sequenceFlow':
            source_ref = element.attrib.get('sourceRef', 'N/A')
            target_ref = element.attrib.get('targetRef', 'N/A')
            print(f"    Source: {source_ref}, Target: {target_ref}")
        elif tag in ['task', 'userTask', 'serviceTask', 'scriptTask']:
            # Example of how to get properties or extensions if they exist
            properties = element.find(f'{NAMESPACES["bpmn"]}extensionElements')
            if properties:
                print(f"    Extension Elements found for {element_id}")
                for prop in properties:
                    print(f"      {prop.tag.replace(NAMESPACES['bpmn'], '')}: {prop.attrib}")

def add_new_task_to_process(root, process_id, new_task_id, new_task_name, prev_task_id, next_task_id):
    """
    Adds a new user task to a process and updates sequence flows.
    This also includes adding corresponding DI elements.
    """
    process = find_element_by_id(root, process_id)
    if not process:
        print(f"Process with ID '{process_id}' not found.")
        return

    # 1. Add the new Task element (semantic model)
    new_task = ET.SubElement(process, f'{NAMESPACES["bpmn"]}userTask', id=new_task_id, name=new_task_name)
    print(f"Added new task: {new_task_id}")

    # 2. Update existing sequence flows and add new ones
    # Find the sequence flow from prev_task_id to next_task_id
    existing_flow = None
    for flow in process.findall(f'{NAMESPACES["bpmn"]}sequenceFlow'):
        if flow.attrib.get('sourceRef') == prev_task_id and flow.attrib.get('targetRef') == next_task_id:
            existing_flow = flow
            break

    if existing_flow:
        # Create a new sequence flow from previous task to new task
        flow1_id = f"Flow_{prev_task_id}_{new_task_id}"
        flow1 = ET.SubElement(process, f'{NAMESPACES["bpmn"]}sequenceFlow', id=flow1_id, sourceRef=prev_task_id, targetRef=new_task_id)
        print(f"Added sequence flow: {flow1_id}")

        # Update the existing sequence flow to go from new task to next task
        # This effectively "inserts" the new task
        existing_flow.set('sourceRef', new_task_id)
        new_flow2_id = f"Flow_{new_task_id}_{next_task_id}"
        existing_flow.set('id', new_flow2_id) # Update the ID of the modified flow as well
        print(f"Updated sequence flow to: {new_flow2_id}")

    else:
        print(f"Warning: Could not find sequence flow from {prev_task_id} to {next_task_id}. New task might not be connected.")

    # 3. Add corresponding DI elements (visual layout)
    bpmn_diagram = root.find(f'.//{NAMESPACES["bpmndi"]}BPMNDiagram')
    if not bpmn_diagram:
        print("Warning: BPMN Diagram not found. Cannot add graphical elements.")
        return

    bpmn_plane = bpmn_diagram.find(f'{NAMESPACES["bpmndi"]}BPMNPlane')
    if not bpmn_plane:
        print("Warning: BPMN Plane not found. Cannot add graphical elements.")
        return

    # Add BPMNShape for the new task
    new_task_shape_id = f"{new_task_id}_di"
    new_task_shape = ET.SubElement(bpmn_plane, f'{NAMESPACES["bpmndi"]}BPMNShape', id=new_task_shape_id, bpmnElement=new_task_id)
    # Position and size - you'll need to calculate these based on surrounding elements
    # For simplicity, let's just place it after the previous task
    prev_task_shape = find_element_by_id(bpmn_plane, f"{prev_task_id}_di")
    x, y, width, height = 0, 0, 100, 80 # Default size
    if prev_task_shape:
        bounds = prev_task_shape.find(f'{NAMESPACES["dc"]}Bounds')
        if bounds is not None:
            prev_x = float(bounds.attrib.get('x', 0))
            prev_y = float(bounds.attrib.get('y', 0))
            prev_width = float(bounds.attrib.get('width', 0))
            # Place the new task 150 units to the right of the previous one
            x = prev_x + prev_width + 150
            y = prev_y

    ET.SubElement(new_task_shape, f'{NAMESPACES["dc"]}Bounds', x=str(x), y=str(y), width=str(width), height=str(height))
    print(f"Added DI shape for new task: {new_task_shape_id}")

    # Add BPMNEdge for the new sequence flows
    if existing_flow:
        # Flow from previous task to new task
        new_flow_1_edge_id = f"{flow1_id}_di"
        flow_1_edge = ET.SubElement(bpmn_plane, f'{NAMESPACES["bpmndi"]}BPMNEdge', id=new_flow_1_edge_id, bpmnElement=flow1_id)
        # Waypoints - simplified, you'd need to calculate these more precisely
        ET.SubElement(flow_1_edge, f'{NAMESPACES["di"]}waypoint', x=str(x - 50), y=str(y + height/2)) # Before new task
        ET.SubElement(flow_1_edge, f'{NAMESPACES["di"]}waypoint', x=str(x), y=str(y + height/2)) # At new task entry
        print(f"Added DI edge for {flow1_id}")

        # Update waypoints for the modified sequence flow (new task to next task)
        # This is more complex as you need to find the old edge and adjust its waypoints.
        # For simplicity, let's remove the old edge and add a new one.
        old_edge_id = f"Flow_{prev_task_id}_{next_task_id}_di" # Assuming original ID
        old_edge = find_element_by_id(bpmn_plane, old_edge_id)
        if old_edge:
            bpmn_plane.remove(old_edge)
            print(f"Removed old DI edge: {old_edge_id}")

        new_flow_2_edge_id = f"{new_flow2_id}_di"
        flow_2_edge = ET.SubElement(bpmn_plane, f'{NAMESPACES["bpmndi"]}BPMNEdge', id=new_flow_2_edge_id, bpmnElement=new_flow2_id)
        ET.SubElement(flow_2_edge, f'{NAMESPACES["di"]}waypoint', x=str(x + width), y=str(y + height/2)) # At new task exit
        ET.SubElement(flow_2_edge, f'{NAMESPACES["di"]}waypoint', x=str(x + width + 50), y=str(y + height/2)) # After new task
        print(f"Added new DI edge for {new_flow2_id}")


def modify_element_attribute(root, element_id, attribute_name, new_value):
    """Modifies an attribute of a specific element by ID."""
    element = find_element_by_id(root, element_id)
    if element:
        element.set(attribute_name, new_value)
        print(f"Modified '{attribute_name}' of '{element_id}' to '{new_value}'")
    else:
        print(f"Element with ID '{element_id}' not found.")

def add_extension_property(root, element_id, property_name, property_value):
    """Adds an extension property to an element."""
    element = find_element_by_id(root, element_id)
    if element:
        # Check for existing extensionElements, create if not present
        extension_elements = element.find(f'{NAMESPACES["bpmn"]}extensionElements')
        if extension_elements is None:
            extension_elements = ET.SubElement(element, f'{NAMESPACES["bpmn"]}extensionElements')
            print(f"Added <extensionElements> to {element_id}")

        # Add the custom property (using a generic 'camunda' namespace for example)
        # You'd typically define a specific namespace for your custom properties.
        ET.SubElement(extension_elements, f'{{http://camunda.org/bpmn/extensions/1.0}}property',
                      name=property_name, value=property_value)
        print(f"Added extension property '{property_name}' to '{element_id}'")
    else:
        print(f"Element with ID '{element_id}' not found.")

def write_bpmn_file(tree, output_file_path):
    """Writes the modified ElementTree back to a BPMN XML file."""
    # Use ET.indent for pretty printing (Python 3.9+)
    ET.indent(tree, space="  ", level=0)
    tree.write(output_file_path, encoding='utf-8', xml_declaration=True, pretty_print=True) # pretty_print works with lxml
    print(f"Modified BPMN written to {output_file_path}")

# --- Main execution ---
if __name__ == "__main__":
    input_file = "my_process.bpmn"
    output_file = "modified_process.bpmn"

    # Create a dummy BPMN file for demonstration if it doesn't exist
    try:
        with open(input_file, 'x') as f:
            f.write("""<?xml version="1.0" encoding="UTF-8"?>
<definitions xmlns="http://www.omg.org/spec/BPMN/20100524/MODEL"
             xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
             xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI"
             xmlns:dc="http://www.omg.org/spec/DD/20100524/DC"
             xmlns:di="http://www.omg.org/spec/DD/20100524/DI"
             id="definitions_1"
             targetNamespace="http://example.com/bpmn">

  <process id="Process_1" isExecutable="true">
    <startEvent id="StartEvent_1" name="Process Started">
      <outgoing>SequenceFlow_1</outgoing>
    </startEvent>
    <task id="Task_A" name="Task A">
      <incoming>SequenceFlow_1</incoming>
      <outgoing>SequenceFlow_2</outgoing>
    </task>
    <task id="Task_B" name="Task B">
      <incoming>SequenceFlow_2</incoming>
      <outgoing>SequenceFlow_3</outgoing>
    </task>
    <endEvent id="EndEvent_1" name="Process Ended">
      <incoming>SequenceFlow_3</incoming>
    </endEvent>
    <sequenceFlow id="SequenceFlow_1" sourceRef="StartEvent_1" targetRef="Task_A" />
    <sequenceFlow id="SequenceFlow_2" sourceRef="Task_A" targetRef="Task_B" />
    <sequenceFlow id="SequenceFlow_3" sourceRef="Task_B" targetRef="EndEvent_1" />
  </process>

  <bpmndi:BPMNDiagram id="BPMNDiagram_1">
    <bpmndi:BPMNPlane id="BPMNPlane_1" bpmnElement="Process_1">
      <bpmndi:BPMNShape id="StartEvent_1_di" bpmnElement="StartEvent_1">
        <dc:Bounds x="179" y="159" width="36" height="36" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Task_A_di" bpmnElement="Task_A">
        <dc:Bounds x="270" y="137" width="100" height="80" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Task_B_di" bpmnElement="Task_B">
        <dc:Bounds x="420" y="137" width="100" height="80" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="EndEvent_1_di" bpmnElement="EndEvent_1">
        <dc:Bounds x="570" y="159" width="36" height="36" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNEdge id="SequenceFlow_1_di" bpmnElement="SequenceFlow_1">
        <di:waypoint x="215" y="177" />
        <di:waypoint x="270" y="177" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_2_di" bpmnElement="SequenceFlow_2">
        <di:waypoint x="370" y="177" />
        <di:waypoint x="420" y="177" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_3_di" bpmnElement="SequenceFlow_3">
        <di:waypoint x="520" y="177" />
        <di:waypoint x="570" y="177" />
      </bpmndi:BPMNEdge>
    </bpmndi:BPMNPlane>
  </bpmndi:BPMNDiagram>

</definitions>
""")
        print(f"Created dummy BPMN file: {input_file}")
    except FileExistsError:
        print(f"Using existing BPMN file: {input_file}")

    tree, root = parse_bpmn_file(input_file)

    print("\nOriginal Process Elements:")
    iterate_and_print_process_elements(root)

    # Example 1: Modify an existing element's attribute
    modify_element_attribute(root, "Task_A", "name", "First Task")

    # Example 2: Add an extension property to a task
    add_extension_property(root, "Task_A", "priority", "high")

    # Example 3: Add a new task in between existing tasks
    add_new_task_to_process(root, "Process_1", "Task_New", "New Intermediate Task", "Task_A", "Task_B")

    print("\nModified Process Elements:")
    iterate_and_print_process_elements(root)

    # Write the modified XML back to a new file
    write_bpmn_file(tree, output_file)

    print("\n--- Verification ---")
    # You can open 'modified_process.bpmn' in a BPMN modeling tool (like Camunda Modeler, Cawemo, draw.io)
    # to visually inspect the changes.
    print(f"Please open '{output_file}' in a BPMN tool to verify the changes.")

In [13]:
import xml.etree.ElementTree as ET

# Define namespaces for easier access
NAMESPACES = {
    'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL',
    'bpmndi': 'http://www.omg.org/spec/BPMN/20100524/DI',
    'dc': 'http://www.omg.org/spec/DD/20100524/DC',
    'ns6': 'http://www.omg.org/spec/DD/20100524/DI'
}


def parse_bpmn_file(file_path):
    """Parses a BPMN XML file and returns the ElementTree root."""
    tree = ET.parse(file_path)
    root = tree.getroot()
    return tree, root

    

def add_sequence_flow_di(bpmn_plane_element, bpmn_flow_id, waypoints):
    """
    Adds the graphical representation (BPMNEdge) for a sequence flow.
    waypoints: A list of (x, y) tuples.
    """
    edge_id = f"{bpmn_flow_id}_di"
    new_edge = ET.SubElement(
        bpmn_plane_element,
        f'{NAMESPACES["bpmndi"]}BPMNEdge',
        id=edge_id,
        bpmnElement=bpmn_flow_id
    )
    for x, y in waypoints:
        ET.SubElement(new_edge, f'{NAMESPACES["di"]}waypoint', x=str(x), y=str(y))
    print(f"Added DI edge for sequenceFlow: {edge_id}")
    return new_edge


def find_elements_by_tag(root, tag_name):
    """Finds all elements with a given tag name, handling namespaces."""
    tags = []
    for item in root.iter():
        if item.tag.endswith("BPMNDiagram"):
            diagram = item
    x_positions = []
    for shape in diagram.iter():
        if shape.tag.endswith("BPMNShape"):
            for bounds in shape.iter():
                if bounds.tag.endswith("Bounds"):
                    x_positions.append(float(bounds.get("x")))
            
    print(min(x_positions))


tree, root = parse_bpmn_file("my_process.bpmn")
find_elements_by_tag(root, "BPMNPlane")
tree.write("res.xml", encoding='utf-8', xml_declaration=True)

100.0
